## Install Packages

In [1]:
!conda install -y -c conda-forge faiss-gpu
!apt-get -y update
!apt-get -y install libatlas-base-dev

done
Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.14.0
  latest version: 25.5.1

Please update conda by running

    $ conda update -n base -c conda-forge conda



# All requested packages already installed.

Retrieving notices: ...working... done
Hit:1 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:2 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:3 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:4 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Reading package lists... Done                     
Reading package lists... Done
Building dependency tree       
Reading state information... Done
libatlas-base-dev is already the newest version (3.10.3-5).
0 upgraded, 0 newly installed, 0 to remove and 83 not upgraded.


## Load Data

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.random_projection import GaussianRandomProjection

from tqdm import tqdm

import faiss

In [3]:
df = pd.read_csv("story_dataset.csv")
df

,prompt_id,prompt,story,hidden_state_file,len_generated_story,len_new_story
0,1,Once upon a time there was a dragon,Once upon a time there was a dragon named Blaz...,./hidden_states/prompt_1.npz,270,271
1,1,Once upon a time there was a dragon,Once upon a time there was a dragon named Spar...,./hidden_states/prompt_1.npz,349,350
2,1,Once upon a time there was a dragon,Once upon a time there was a dragon named Scor...,./hidden_states/prompt_1.npz,278,278
3,1,Once upon a time there was a dragon,Once upon a time there was a dragon. The drago...,./hidden_states/prompt_1.npz,117,118
4,1,Once upon a time there was a dragon,Once upon a time there was a dragon. The drago...,./hidden_states/prompt_1.npz,129,130
...,...,...,...,...,...,...
9995,10,Once upon a time there was a poor boy,Once upon a time there was a poor boy named Ti...,./hidden_states/prompt_10.npz,289,290
9996,10,Once upon a time there was a poor boy,Once upon a time there was a poor boy named Ti...,./hidden_states/prompt_10.npz,119,119
9997,10,Once upon a time there was a poor boy,Once upon a time there was a poor boy named Ti...,./hidden_states/prompt_10.npz,127,128
9998,10,Once upon a time there was a poor boy,Once upon a time there was a poor boy named Ti...,./hidden_states/prompt_10.npz,441,441


In [4]:
max_story_len = max(df["len_generated_story"])
max_story_len

522

In [5]:
hidden_states_by_layer = {}
NUM_PROMPTS = 10

for prompt_id in range(1, NUM_PROMPTS + 1):
    with np.load(f'./hidden_states/prompt_{prompt_id}.npz') as loaded_data:
        for i in tqdm(range(1000)):
            curr_hidden_states = loaded_data[f"arr_{i}"][0]
#             print(curr_hidden_states.shape)
            for layer in range(7, 8):
                padded_arr = np.zeros((max_story_len, 512))
                padded_arr_len = len(curr_hidden_states[layer][0])
                
                padded_arr[:padded_arr_len] = curr_hidden_states[layer][0]
                
                padded_arr = padded_arr.flatten().astype('float32') #FAISS expects data in type float32 instead of float64 - saves memory too!
#                 print(padded_arr.shape)
                
                if(f"layer_{layer}" in hidden_states_by_layer):
                    hidden_states_by_layer[f"layer_{layer}"].append(padded_arr)
                else:
                    hidden_states_by_layer[f"layer_{layer}"] = [padded_arr]

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1000/1000 [02:35<00:00,  6.41it/s]


In [6]:
layer_hs_array = np.array(hidden_states_by_layer["layer_7"])
layer_hs_array.shape

(10000, 267264)

## Layer 7 Clustering

In [7]:
# Use original vectors for clustering - uncomment next line and comment out last two lines

# dim_reduced_vecs = layer_hs_array

random_projector = GaussianRandomProjection(random_state = 42)
dim_reduced_vecs = random_projector.fit_transform(layer_hs_array).astype('float32')

In [8]:
dim_reduced_vecs = np.array([v / np.linalg.norm(v) for v in dim_reduced_vecs])
dim_reduced_vecs.shape

(10000, 7894)

In [9]:
# K-means Clustering

ncentroids = NUM_PROMPTS
niter = 20
verbose = True
dim = dim_reduced_vecs.shape[1]
kmeans = faiss.Kmeans(dim, ncentroids, niter = niter, verbose = verbose, gpu = True, nredo = 10, spherical = True, max_points_per_centroid = 1000)
kmeans.train(dim_reduced_vecs)

Clustering 10000 points in 7894D to 10 clusters, redo 10 times, 20 iterations
  Preprocessing in 0.06 s
Outer iteration 0 / 10
  Iteration 19 (1.65 s, search 1.05 s): objective=5075.41 imbalance=1.319 nsplit=0       
Objective improved: keep new clusters
Outer iteration 1 / 10
  Iteration 19 (3.31 s, search 2.10 s): objective=5088.81 imbalance=1.339 nsplit=0       
Objective improved: keep new clusters
Outer iteration 2 / 10
  Iteration 19 (4.73 s, search 2.99 s): objective=5086.91 imbalance=1.221 nsplit=0       
Outer iteration 3 / 10
  Iteration 19 (6.17 s, search 3.90 s): objective=5080.75 imbalance=1.268 nsplit=0       
Outer iteration 4 / 10
  Iteration 19 (7.59 s, search 4.78 s): objective=5068.79 imbalance=1.386 nsplit=0       
Outer iteration 5 / 10
  Iteration 19 (9.01 s, search 5.67 s): objective=5086.98 imbalance=1.244 nsplit=0       
Outer iteration 6 / 10
  Iteration 19 (10.42 s, search 6.55 s): objective=5085.33 imbalance=1.302 nsplit=0       
Outer iteration 7 / 10
  Ite

5088.8076171875

In [10]:
kmeans.centroids #cluster centers

array([[ 0.00979454, -0.00673794,  0.02064081, ...,  0.00161652,
        -0.00210845, -0.00301413],
       [ 0.00506898, -0.0140111 ,  0.00593833, ..., -0.01155534,
        -0.00508402,  0.00401272],
       [ 0.01066381, -0.01334879,  0.01758316, ...,  0.00193596,
        -0.00218558,  0.00611459],
       ...,
       [ 0.00695984, -0.01258845,  0.01655486, ..., -0.00148269,
        -0.01465738,  0.00682859],
       [ 0.00360913, -0.02045013,  0.01212575, ..., -0.01574161,
        -0.00538365,  0.00569591],
       [ 0.00571735, -0.01079431,  0.00669099, ..., -0.00875144,
        -0.00292508,  0.01123711]], dtype=float32)

In [11]:
for centroid in kmeans.centroids:
    print(np.linalg.norm(centroid))

0.99999994
0.99999994
0.9999999
1.0000002
1.0000001
0.9999997
1.0
1.0
0.99999994
0.99999976


In [12]:
kmeans.obj #inertia at each iteration

array([2814.06982422, 4919.23144531, 4989.83740234, 5036.20654297,
       5061.33447266, 5066.09423828, 5068.73046875, 5071.42431641,
       5073.06494141, 5074.19433594, 5074.79101562, 5075.06884766,
       5075.23925781, 5075.29736328, 5075.31884766, 5075.34521484,
       5075.36914062, 5075.390625  , 5075.40087891, 5075.40917969,
       2911.98413086, 4919.45458984, 5045.18945312, 5067.09667969,
       5076.36914062, 5081.27636719, 5087.26269531, 5088.17724609,
       5088.50830078, 5088.61279297, 5088.68945312, 5088.75341797,
       5088.78808594, 5088.79101562, 5088.79882812, 5088.80566406,
       5088.80712891, 5088.80761719, 5088.80761719, 5088.80761719])

In [13]:
pd.Series(kmeans.index.search(dim_reduced_vecs.astype(np.float32), 1)[1].flatten()).value_counts()

4    2105
1    1886
6    1276
9    1188
5    1041
8     605
0     554
2     511
3     424
7     410
dtype: int64

In [14]:
normalized_vecs = [v / np.linalg.norm(v) for v in dim_reduced_vecs]

In [15]:
cos_similarities = normalized_vecs @ kmeans.centroids.T
classifications = np.argmax(cos_similarities, axis=1)

In [16]:
pd.Series(classifications).value_counts()

4    2105
1    1886
6    1276
9    1188
5    1041
8     605
0     554
2     511
3     424
7     410
dtype: int64